In [1]:
from pyspark.sql import SparkSession, functions as F
import os

In [2]:
spark = (
    SparkSession
    .builder
    # .master("spark://spark-master:7077")
    .appName("Loading cleaned files")
    .config("spark.jars", "https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-2.2.14.jar") # GCS Connector
    .getOrCreate()
)

# Google Cloud Service Account Credentials
spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile",os.environ.get("GOOGLE_APPLICATION_CREDENTIALS"))

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/04 10:39:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
bucket='gs://zoomcamp-454219-ade-pipeline/data/pq/'

In [4]:
df = (spark.read.parquet(bucket+"drug/2005/drug-event-part-15-of-21.parquet"))
df.printSchema()

root
 |-- patientid: string (nullable = true)
 |-- medicinalproduct: string (nullable = true)
 |-- activesubstancename: string (nullable = true)
 |-- drugindication: string (nullable = true)
 |-- drugadministrationroute: string (nullable = true)
 |-- drugstartdate: string (nullable = true)
 |-- drugenddate: string (nullable = true)
 |-- drugdosagetext: string (nullable = true)
 |-- drugstructuredosagenumb: string (nullable = true)
 |-- drugstructuredosageunit: string (nullable = true)
 |-- drugtreatmentduration: string (nullable = true)
 |-- drugtreatmentdurationunit: string (nullable = true)
 |-- drugrecurreadministration: string (nullable = true)



In [5]:
(
    df
    .select('drugstartdate')
    .filter((F.col('drugstartdate').isNotNull()))
    .groupby(
        F
        .year(F.col('drugstartdate').substr(0,4))
        .alias("YEAR")
    )
    .count()
    .select("YEAR")
    .distinct()
    .orderBy(F.asc("YEAR"))
    .show()
)

+----+
|YEAR|
+----+
|1004|
|1945|
|1955|
|1958|
|1960|
|1961|
|1962|
|1964|
|1967|
|1969|
|1970|
|1971|
|1972|
|1973|
|1974|
|1975|
|1976|
|1978|
|1979|
|1980|
+----+
only showing top 20 rows



In [6]:
df.filter((F.col('drugstartdate').isNotNull()) & (F.col('drugstartdate')==10041227)).show()

+--------------------+----------------+-------------------+------------------+-----------------------+-------------+-----------+--------------+-----------------------+-----------------------+---------------------+-------------------------+-------------------------+
|           patientid|medicinalproduct|activesubstancename|    drugindication|drugadministrationroute|drugstartdate|drugenddate|drugdosagetext|drugstructuredosagenumb|drugstructuredosageunit|drugtreatmentduration|drugtreatmentdurationunit|drugrecurreadministration|
+--------------------+----------------+-------------------+------------------+-----------------------+-------------+-----------+--------------+-----------------------+-----------------------+---------------------+-------------------------+-------------------------+
|d3ae45d3-9a40-4e5...|         ABILIFY|               null|BIPOLAR I DISORDER|                    048|     10041227|       null|          null|                   null|                   null|           

In [8]:
df1 = spark.read.parquet(bucket+"patient/*/*")
df1.select('receivedate').distinct().show()

+-----------+
|receivedate|
+-----------+
|       null|
+-----------+

